<a href="https://colab.research.google.com/github/gomlfx/ApiaryFund/blob/master/eurusd_automl_hist_range.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install flaml

In [2]:
#designed for Jupyter/kaggle/colab
import pandas as pd
import numpy as np
#import matplotlib for plotting 
import matplotlib.pyplot as plt
%matplotlib inline
#import normalization
from sklearn import preprocessing
import xgboost as xg
from sklearn.ensemble import RandomForestRegressor as rf
#import auto hyperparameter tuning
from flaml import AutoML
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from lightgbm import LGBMRegressor


In [8]:
 #MT4 csv 
df1=pd.read_csv('EURUSD1440.csv', names=['d','t','eu_o','eu_h','eu_l','eu_c','eu_v'])
df_m_5 = df1.tail(2000)
df2 = df1.tail(2000)
# df2=pd.read_csv('GBPUSD1440.csv', names=['d','t','gu_o','gu_h','gu_l','gu_c','gu_v'])
# df2 = df2.tail(2000)

# df3=pd.read_csv('USDCAD1440.csv', names=['d','t','uc_o','uc_h','uc_l','uc_c','uc_v'])
# df3 = df3.tail(2000)

# df4=pd.read_csv('USDCHF1440.csv', names=['d','t','uf_o','uf_h','uf_l','uf_c','uf_v'])
# df4 = df4.tail(2000)

# df5=pd.read_csv('USDJPY1440.csv', names=['d','t','uj_o','uj_h','uj_l','uj_c','uj_v'])
# df5 = df5.tail(2000)

# df6=pd.read_csv('USDSEK1440.csv', names=['d','t','us_o','us_h','us_l','us_c','us_v'])
# df6 = df6.tail(2000)

# #using merge
# df_m_1 = df1.merge(df2, on='d')
# df_m_2 = df_m_1.merge(df3, on='d')
# df_m_3 = df_m_2.merge(df4, on='d')
# df_m_4 = df_m_3.merge(df5, on='d')
# df_m_5 = df_m_4.merge(df6, on='d')
# df_m_5 = df_m_5.drop(columns='t_x')
# df_m_5 = df_m_5.drop(columns='t_y')

# pd.set_option('display.max_columns', None)
# print(df_m_5) 

In [9]:
df2.tail()

,d,t,eu_o,eu_h,eu_l,eu_c,eu_v
12784,2021.12.06,00:00,1.12997,1.13109,1.12664,1.12848,79067
12785,2021.12.07,00:00,1.12821,1.12982,1.12273,1.12681,78783
12786,2021.12.08,00:00,1.12650,1.13547,1.12627,1.13418,85355
12787,2021.12.09,00:00,1.13422,1.13452,1.12782,1.12924,71564
12788,2021.12.10,00:00,1.12913,1.12980,1.12868,1.12964,4690


In [10]:

# #2 days forward
# df_m_5['eu_High_next_day2'] = df_m_5['eu_h'].shift(-2)
# df_m_5['eu_Low_next_day2'] = df_m_5['eu_l'].shift(-2)
# df_m_5['eu_Close_next_day2'] = df_m_5['eu_c'].shift(-2) 
# #3 days forward
# df_m_5['eu_High_next_day3'] = df_m_5['eu_h'].shift(-3)
# df_m_5['eu_Low_next_day3'] = df_m_5['eu_l'].shift(-3)
# df_m_5['eu_Close_next_day3'] = df_m_5['eu_c'].shift(-3) 
# #4 days forward 
# df_m_5['eu_High_next_day4'] = df_m_5['eu_h'].shift(-4)
# df_m_5['eu_Low_next_day4'] = df_m_5['eu_l'].shift(-4)
# df_m_5['eu_Close_next_day4'] = df_m_5['eu_c'].shift(-4) 
# #5 days forward
# df_m_5['eu_High_next_day5'] = df_m_5['eu_h'].shift(-5)
# df_m_5['eu_Low_next_day5'] = df_m_5['eu_l'].shift(-5)
# df_m_5['eu_Close_next_day5'] = df_m_5['eu_c'].shift(-5) 

#Features
#1 day ago (features)
df_m_5['eu_High_last_day1'] = df_m_5['eu_h'].shift(1)
df_m_5['eu_Low_last_day1'] = df_m_5['eu_l'].shift(1)
df_m_5['eu_Close_last_day1'] = df_m_5['eu_c'].shift(1) 
#2 days ago
df_m_5['eu_High_last_day2'] = df_m_5['eu_h'].shift(2)
df_m_5['eu_Low_last_day2'] = df_m_5['eu_l'].shift(2)
df_m_5['eu_Close_last_day2'] = df_m_5['eu_c'].shift(2) 
#3 days ago
df_m_5['eu_High_last_day3'] = df_m_5['eu_h'].shift(3)
df_m_5['eu_Low_last_day3'] = df_m_5['eu_l'].shift(3)
df_m_5['eu_Close_last_day3'] = df_m_5['eu_c'].shift(3) 
#4 days ago 
df_m_5['eu_High_last_day4'] = df_m_5['eu_h'].shift(4)
df_m_5['eu_Low_last_day4'] = df_m_5['eu_l'].shift(4)
df_m_5['eu_Close_last_day4'] = df_m_5['eu_c'].shift(4) 
#5 days ago
df_m_5['eu_High_last_day5'] = df_m_5['eu_h'].shift(5)
df_m_5['eu_Low_last_day5'] = df_m_5['eu_l'].shift(5)
df_m_5['eu_Close_last_day5'] = df_m_5['eu_c'].shift(5) 

#Target
#1 day forward 
df_m_5['eu_High_next_day1'] = df_m_5['eu_h'].shift(-1)
df_m_5['eu_Low_next_day1'] = df_m_5['eu_l'].shift(-1)
df_m_5['eu_Close_next_day1'] = df_m_5['eu_c'].shift(-1) 

#drop null and useless columns
df_m_5 = df_m_5.dropna()
df_m_5 = df_m_5.drop(['t','eu_v'], axis=1)



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

In [11]:
df_m_5 

,d,eu_o,eu_h,eu_l,eu_c,eu_High_last_day1,eu_Low_last_day1,eu_Close_last_day1,eu_High_last_day2,eu_Low_last_day2,eu_Close_last_day2,eu_High_last_day3,eu_Low_last_day3,eu_Close_last_day3,eu_High_last_day4,eu_Low_last_day4,eu_Close_last_day4,eu_High_last_day5,eu_Low_last_day5,eu_Close_last_day5,eu_High_next_day1,eu_Low_next_day1,eu_Close_next_day1
10794,2014.04.03,1.37653,1.38066,1.36980,1.37168,1.38201,1.37530,1.37651,1.38152,1.37690,1.37932,1.38085,1.37215,1.37735,1.37729,1.37047,1.37513,1.37967,1.37283,1.37437,1.37312,1.36725,1.37014
10795,2014.04.04,1.37169,1.37312,1.36725,1.37014,1.38066,1.36980,1.37168,1.38201,1.37530,1.37651,1.38152,1.37690,1.37932,1.38085,1.37215,1.37735,1.37729,1.37047,1.37513,1.37483,1.36960,1.37412
10796,2014.04.07,1.37008,1.37483,1.36960,1.37412,1.37312,1.36725,1.37014,1.38066,1.36980,1.37168,1.38201,1.37530,1.37651,1.38152,1.37690,1.37932,1.38085,1.37215,1.37735,1.38116,1.37371,1.37946
10797,2014.04.08,1.37413,1.38116,1.37371,1.37946,1.37483,1.36960,1.37412,1.37312,1.36725,1.37014,1.38066,1.36980,1.37168,1.38201,1.37530,1.37651,1.38152,1.37690,1.37932,1.38620,1.37799,1.38536
10798,2014.04.09,1.37955,1.38620,1.37799,1.38536,1.38116,1.37371,1.37946,1.37483,1.36960,1.37412,1.37312,1.36725,1.37014,1.38066,1.36980,1.37168,1.38201,1.37530,1.37651,1.38988,1.38357,1.38868
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12783,2021.12.03,1.13005,1.13333,1.12665,1.13088,1.13475,1.12952,1.13013,1.13594,1.13023,1.13190,1.13827,1.12350,1.13390,1.13117,1.12582,1.12918,1.13295,1.12051,1.13177,1.13109,1.12664,1.12848
12784,2021.12.06,1.12997,1.13109,1.12664,1.12848,1.13333,1.12665,1.13088,1.13475,1.12952,1.13013,1.13594,1.13023,1.13190,1.13827,1.12350,1.13390,1.13117,1.12582,1.12918,1.12982,1.12273,1.12681
12785,2021.12.07,1.12821,1.12982,1.12273,1.12681,1.13109,1.12664,1.12848,1.13333,1.12665,1.13088,1.13475,1.12952,1.13013,1.13594,1.13023,1.13190,1.13827,1.12350,1.13390,1.13547,1.12627,1.13418
12786,2021.12.08,1.12650,1.13547,1.12627,1.13418,1.12982,1.12273,1.12681,1.13109,1.12664,1.12848,1.13333,1.12665,1.13088,1.13475,1.12952,1.13013,1.13594,1.13023,1.13190,1.13452,1.12782,1.12924


In [12]:
df_m_5.insert(1,'index',df_m_5.index)

In [ ]:
# #new column: high vs low
# df_m_5['eu_h_or_l'] = ((df_m_5.eu_h - df_m_5.eu_o) > (df_m_5.eu_o - df_m_5.eu_l)) 
# df_m_5.eu_h_or_l = df_m_5.eu_h_or_l.replace({True:1,False:0})

# #new column: close vs yesterday
# df_m_5['eu_c_vs_c'] = (df_m_5.eu_c > df_m_5.eu_o) 
# df_m_5.eu_c_vs_c = df_m_5.eu_c_vs_c.replace({True:1,False:0})

# #new column: shift tomorrow to today
# df_m_5['eu_High_next_day'] = df_m_5['eu_h'].shift(-1)
# df_m_5['eu_Low_next_day'] = df_m_5['eu_l'].shift(-1)
# df_m_5['eu_Close_next_day'] = df_m_5['eu_c'].shift(-1)
# df_m_5 = df_m_5.dropna()


In [13]:
df_m_5.tail()

,d,index,eu_o,eu_h,eu_l,eu_c,eu_High_last_day1,eu_Low_last_day1,eu_Close_last_day1,eu_High_last_day2,eu_Low_last_day2,eu_Close_last_day2,eu_High_last_day3,eu_Low_last_day3,eu_Close_last_day3,eu_High_last_day4,eu_Low_last_day4,eu_Close_last_day4,eu_High_last_day5,eu_Low_last_day5,eu_Close_last_day5,eu_High_next_day1,eu_Low_next_day1,eu_Close_next_day1
12783,2021.12.03,12783,1.13005,1.13333,1.12665,1.13088,1.13475,1.12952,1.13013,1.13594,1.13023,1.13190,1.13827,1.12350,1.13390,1.13117,1.12582,1.12918,1.13295,1.12051,1.13177,1.13109,1.12664,1.12848
12784,2021.12.06,12784,1.12997,1.13109,1.12664,1.12848,1.13333,1.12665,1.13088,1.13475,1.12952,1.13013,1.13594,1.13023,1.13190,1.13827,1.12350,1.13390,1.13117,1.12582,1.12918,1.12982,1.12273,1.12681
12785,2021.12.07,12785,1.12821,1.12982,1.12273,1.12681,1.13109,1.12664,1.12848,1.13333,1.12665,1.13088,1.13475,1.12952,1.13013,1.13594,1.13023,1.13190,1.13827,1.12350,1.13390,1.13547,1.12627,1.13418
12786,2021.12.08,12786,1.12650,1.13547,1.12627,1.13418,1.12982,1.12273,1.12681,1.13109,1.12664,1.12848,1.13333,1.12665,1.13088,1.13475,1.12952,1.13013,1.13594,1.13023,1.13190,1.13452,1.12782,1.12924
12787,2021.12.09,12787,1.13422,1.13452,1.12782,1.12924,1.13547,1.12627,1.13418,1.12982,1.12273,1.12681,1.13109,1.12664,1.12848,1.13333,1.12665,1.13088,1.13475,1.12952,1.13013,1.12980,1.12868,1.12964


In [14]:

df_m_5.isna().sum()

d                     0
index                 0
eu_o                  0
eu_h                  0
eu_l                  0
eu_c                  0
eu_High_last_day1     0
eu_Low_last_day1      0
eu_Close_last_day1    0
eu_High_last_day2     0
eu_Low_last_day2      0
eu_Close_last_day2    0
eu_High_last_day3     0
eu_Low_last_day3      0
eu_Close_last_day3    0
eu_High_last_day4     0
eu_Low_last_day4      0
eu_Close_last_day4    0
eu_High_last_day5     0
eu_Low_last_day5      0
eu_Close_last_day5    0
eu_High_next_day1     0
eu_Low_next_day1      0
eu_Close_next_day1    0
dtype: int64

In [15]:
df_m_5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1994 entries, 10794 to 12787
Data columns (total 24 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   d                   1994 non-null   object 
 1   index               1994 non-null   int64  
 2   eu_o                1994 non-null   float64
 3   eu_h                1994 non-null   float64
 4   eu_l                1994 non-null   float64
 5   eu_c                1994 non-null   float64
 6   eu_High_last_day1   1994 non-null   float64
 7   eu_Low_last_day1    1994 non-null   float64
 8   eu_Close_last_day1  1994 non-null   float64
 9   eu_High_last_day2   1994 non-null   float64
 10  eu_Low_last_day2    1994 non-null   float64
 11  eu_Close_last_day2  1994 non-null   float64
 12  eu_High_last_day3   1994 non-null   float64
 13  eu_Low_last_day3    1994 non-null   float64
 14  eu_Close_last_day3  1994 non-null   float64
 15  eu_High_last_day4   1994 non-null   float64
 16  e

In [17]:
#extract row for variable
last_row = df_m_5.iloc[-1:,np.r_[2:21]].values
print(last_row) 

[[1.13422 1.13452 1.12782 1.12924 1.13547 1.12627 1.13418 1.12982 1.12273
  1.12681 1.13109 1.12664 1.12848 1.13333 1.12665 1.13088 1.13475 1.12952
  1.13013]]


In [26]:
#for high prediction
#select rows to use for x,y
x_h = df_m_5.iloc[:,np.r_[2:21]].values
print(x_h)
y_h = df_m_5.iloc[:,[21]].values
print(y_h) 

[[1.37653 1.38066 1.3698  ... 1.37967 1.37283 1.37437]
 [1.37169 1.37312 1.36725 ... 1.37729 1.37047 1.37513]
 [1.37008 1.37483 1.3696  ... 1.38085 1.37215 1.37735]
 ...
 [1.12821 1.12982 1.12273 ... 1.13827 1.1235  1.1339 ]
 [1.1265  1.13547 1.12627 ... 1.13594 1.13023 1.1319 ]
 [1.13422 1.13452 1.12782 ... 1.13475 1.12952 1.13013]]
[[1.37312]
 [1.37483]
 [1.38116]
 ...
 [1.13547]
 [1.13452]
 [1.1298 ]]


In [27]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_h, y_h, test_size = 0.30)

In [28]:
automl_reg = AutoML()
automl_reg.fit(X_train, y_train, estimator_list="auto", task="regression", time_budget=None, eval_method="auto", n_jobs=-1)

[flaml.automl: 12-10 03:50:49] {1943} INFO - task = regression
[flaml.automl: 12-10 03:50:49] {1945} INFO - Data split method: uniform
[flaml.automl: 12-10 03:50:49] {1949} INFO - Evaluation method: cv
[flaml.automl: 12-10 03:50:49] {2019} INFO - Minimizing error metric: 1-r2
[flaml.automl: 12-10 03:50:49] {2071} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'xgboost', 'extra_tree', 'xgb_limitdepth']
[flaml.automl: 12-10 03:50:49] {2311} INFO - iteration 0, current learner lgbm
[flaml.automl: 12-10 03:50:50] {2425} INFO - Estimated sufficient time budget=583s. Estimated necessary time budget=4s.
[flaml.automl: 12-10 03:50:50] {2505} INFO -  at 0.1s,	estimator lgbm's best error=0.4805,	best estimator lgbm's best error=0.4805
[flaml.automl: 12-10 03:50:50] {2311} INFO - iteration 1, current learner lgbm
[flaml.automl: 12-10 03:50:50] {2505} INFO -  at 0.1s,	estimator lgbm's best error=0.4805,	best estimator lgbm's best error=0.4805
[flaml.automl: 12-10 03:50:50] {2311} INFO - 

In [29]:
#paste retrained model
sugg_reg_high_param = LGBMRegressor(colsample_bytree=0.8048852608933502,
              learning_rate=0.12880577181319283, max_bin=1023,
              min_child_samples=4, n_estimators=61, num_leaves=16,
              reg_alpha=0.007704104902643932, reg_lambda=0.013155944675837352,
              verbose=-1)
#paste finished
sugg_reg_high_param.fit(X_train,y_train)
y_pred_h=sugg_reg_high_param.predict(last_row)
y_pred_h

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([1.13515318])

In [30]:
y_pred = sugg_reg_high_param.predict(X_test)

In [31]:
def mean_absolute_percentage_error(y_true,y_pred):
  y_true, y_pred = np.array(y_true), np.array(y_pred)
  return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mape = mean_absolute_percentage_error(y_test,y_pred)

In [32]:
#for regression score
from sklearn import metrics 
print('MAE:', metrics.mean_absolute_error(y_test,y_pred))
print('MSE:', metrics.mean_squared_error(y_test, y_pred))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('MAPE:', mape)
print('MedAE', metrics.median_absolute_error(y_test,y_pred)) 

MAE: 0.003111387670970527
MSE: 1.62916622254228e-05
RMSE: 0.004036293129273789
MAPE: 6.147222551024925
MedAE 0.002575680853954543


In [33]:
#for low prediction
#select rows to use for x,y
x_l = df_m_5.iloc[:,np.r_[2:21]].values
print(x_l)
y_l = df_m_5.iloc[:,[22]].values
print(y_l) 

[[1.37653 1.38066 1.3698  ... 1.37967 1.37283 1.37437]
 [1.37169 1.37312 1.36725 ... 1.37729 1.37047 1.37513]
 [1.37008 1.37483 1.3696  ... 1.38085 1.37215 1.37735]
 ...
 [1.12821 1.12982 1.12273 ... 1.13827 1.1235  1.1339 ]
 [1.1265  1.13547 1.12627 ... 1.13594 1.13023 1.1319 ]
 [1.13422 1.13452 1.12782 ... 1.13475 1.12952 1.13013]]
[[1.36725]
 [1.3696 ]
 [1.37371]
 ...
 [1.12627]
 [1.12782]
 [1.12868]]


In [34]:
#for low prediction
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_l, y_l, test_size = 0.30)

In [35]:
automl_reg = AutoML()
automl_reg.fit(X_train, y_train, estimator_list="auto", task="regression", time_budget=None, eval_method="auto", n_jobs=-1)

[flaml.automl: 12-10 03:52:19] {1943} INFO - task = regression
[flaml.automl: 12-10 03:52:19] {1945} INFO - Data split method: uniform
[flaml.automl: 12-10 03:52:19] {1949} INFO - Evaluation method: cv
[flaml.automl: 12-10 03:52:19] {2019} INFO - Minimizing error metric: 1-r2
[flaml.automl: 12-10 03:52:19] {2071} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'xgboost', 'extra_tree', 'xgb_limitdepth']
[flaml.automl: 12-10 03:52:19] {2311} INFO - iteration 0, current learner lgbm
[flaml.automl: 12-10 03:52:19] {2425} INFO - Estimated sufficient time budget=505s. Estimated necessary time budget=4s.
[flaml.automl: 12-10 03:52:19] {2505} INFO -  at 0.1s,	estimator lgbm's best error=0.4781,	best estimator lgbm's best error=0.4781
[flaml.automl: 12-10 03:52:19] {2311} INFO - iteration 1, current learner lgbm
[flaml.automl: 12-10 03:52:19] {2505} INFO -  at 0.1s,	estimator lgbm's best error=0.4781,	best estimator lgbm's best error=0.4781
[flaml.automl: 12-10 03:52:19] {2311} INFO - 

In [ ]:
#paste retrained model
sugg_reg_low_param = ExtraTreesRegressor(max_features=0.5696184314663663, max_leaf_nodes=147,
                    n_estimators=14, n_jobs=-1)
#paste finished
sugg_reg_low_param.fit(X_train,y_train)
y_pred_l=sugg_reg_low_param.predict(last_row)
y_pred_l

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


array([1.12702482])

In [ ]:
#for low pred 
y_pred = sugg_reg_low_param.predict(X_test)
mape = mean_absolute_percentage_error(y_test,y_pred)

In [ ]:
#for low pred regression score
from sklearn import metrics 
print('MAE:', metrics.mean_absolute_error(y_test,y_pred))
print('MSE:', metrics.mean_squared_error(y_test, y_pred))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('MAPE:', mape)
print('MedAE', metrics.median_absolute_error(y_test,y_pred)) 

MAE: 0.003156698822482405
MSE: 1.809548119031102e-05
RMSE: 0.0042538783704181085
MAPE: 6.589793022770839
MedAE 0.002520567653878425


In [ ]:
#for close prediction
#select rows to use for x,y
x_c = df_m_5.iloc[:,np.r_[2:21]].values
print(x_c)
y_c = df_m_5.iloc[:,[23]].values
print(y_c) 

[[0.00000e+00 1.37834e+00 1.37967e+00 ... 5.89320e+04 0.00000e+00
  0.00000e+00]
 [1.00000e+00 1.37436e+00 1.37729e+00 ... 4.74900e+04 0.00000e+00
  1.00000e+00]
 [2.00000e+00 1.37531e+00 1.38085e+00 ... 6.11360e+04 1.00000e+00
  1.00000e+00]
 ...
 [1.99400e+03 1.12821e+00 1.12982e+00 ... 3.42490e+04 0.00000e+00
  0.00000e+00]
 [1.99500e+03 1.12650e+00 1.13547e+00 ... 3.27370e+04 1.00000e+00
  1.00000e+00]
 [1.99600e+03 1.13422e+00 1.13452e+00 ... 3.27310e+04 0.00000e+00
  0.00000e+00]]
[[1.37513]
 [1.37735]
 [1.37932]
 ...
 [1.13418]
 [1.12924]
 [1.12964]]


In [ ]:
#for close prediction
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_c, y_c, test_size = 0.30)

In [ ]:
automl_reg = AutoML()
automl_reg.fit(X_train, y_train, estimator_list="auto", task="regression", time_budget=None, eval_method="auto")

[flaml.automl: 12-10 02:21:14] {1943} INFO - task = regression
[flaml.automl: 12-10 02:21:14] {1945} INFO - Data split method: uniform
[flaml.automl: 12-10 02:21:14] {1949} INFO - Evaluation method: cv
[flaml.automl: 12-10 02:21:14] {2019} INFO - Minimizing error metric: 1-r2
[flaml.automl: 12-10 02:21:14] {2071} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'xgboost', 'extra_tree', 'xgb_limitdepth']
[flaml.automl: 12-10 02:21:14] {2311} INFO - iteration 0, current learner lgbm
[flaml.automl: 12-10 02:21:14] {2425} INFO - Estimated sufficient time budget=883s. Estimated necessary time budget=6s.
[flaml.automl: 12-10 02:21:14] {2505} INFO -  at 0.1s,	estimator lgbm's best error=0.4764,	best estimator lgbm's best error=0.4764
[flaml.automl: 12-10 02:21:14] {2311} INFO - iteration 1, current learner lgbm
[flaml.automl: 12-10 02:21:15] {2505} INFO -  at 0.2s,	estimator lgbm's best error=0.4764,	best estimator lgbm's best error=0.4764
[flaml.automl: 12-10 02:21:15] {2311} INFO - 

In [ ]:
#paste retrained model
sugg_reg_close_param = ExtraTreesRegressor(max_features=0.5214606300593904, max_leaf_nodes=89,
                    n_estimators=41, n_jobs=-1)
#paste finished
sugg_reg_close_param.fit(X_train,y_train)
y_pred_c=sugg_reg_close_param.predict(last_row)
y_pred_c

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


array([1.12971246])

In [ ]:
#for close prediction
y_pred = sugg_reg_close_param.predict(X_test)
mape = mean_absolute_percentage_error(y_test,y_pred)

In [ ]:
#for close regression score
from sklearn import metrics 
print('MAE:', metrics.mean_absolute_error(y_test,y_pred))
print('MSE:', metrics.mean_squared_error(y_test, y_pred))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('MAPE:', mape)
print('MedAE', metrics.median_absolute_error(y_test,y_pred)) 
#print('MdAPE', metrics)

MAE: 0.004253417680207975
MSE: 3.197885401776454e-05
RMSE: 0.00565498488218709
MAPE: 5.800026649654338
MedAE 0.003055985845086484


In [ ]:
print('Today the predictions are:')
print('high', y_pred_h)
print('close', y_pred_c)
print('low', y_pred_l)
print('Best used as polarities.')

Today the predictions are:
high [1.13345194]
close [1.12997479]
low [1.1271146]
Best used as polarities.
